In [ ]:
%load_ext autoreload
%autoreload 2

import logging

import matplotlib as mpl
import matplotlib.pyplot as plt

import defi.dao_cross.order as ddacrord
import defi.dao_cross.supply_demand as ddcrsede
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import defi.tulip.implementation.order as dtuimor
import defi.tulip.implementation.supply_demand as dtuimsd

In [ ]:
try:
    import pulp
except ImportError:
    !sudo /bin/bash -c "(source /venv/bin/activate; pip install pulp)"
    import pulp
import defi.tulip.implementation.optimize as dtuimopt

In [ ]:
hdbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

# Get aggregated supply / demand

In [ ]:
type_ = "supply"
alpha = 2.0
beta = 10.0
n_orders = 10
agg_supply_curve = dtuimsd.get_supply_demand_aggregated_curve(
    type_, alpha, beta, n_orders
)
agg_supply_curve

In [ ]:
type_ = "demand"
alpha = -2.0
beta = 210.0
n_orders = 10
agg_demand_curve = dtuimsd.get_supply_demand_aggregated_curve(
    type_, alpha, beta, n_orders
)
agg_demand_curve

In [ ]:
plt.plot(agg_supply_curve)
plt.plot(agg_demand_curve)
plt.show()

In [ ]:
agg_supply_orders = dtuimsd.convert_aggregated_curve_to_limit_orders(agg_supply_curve)
agg_supply_orders_df = dtuimor.convert_orders_to_dataframe(agg_supply_orders)
agg_supply_orders_df

In [ ]:
agg_demand_orders = dtuimsd.convert_aggregated_curve_to_limit_orders(agg_demand_curve)
agg_demand_orders_df = dtuimor.convert_orders_to_dataframe(agg_demand_orders)
agg_demand_orders_df

# Implement DaoCross and DaoSwap

In [ ]:
agg_all_orders = agg_supply_orders + agg_demand_orders
prices = {"BTC": 1, "ETH": 50}

In [ ]:
daocross_results = dtuimopt.run_daocross_solver(agg_all_orders, prices)
display(daocross_results)

In [ ]:
daoswap_results = dtuimopt.run_daoswap_solver(agg_all_orders)
display(daoswap_results)

# Sanity check with discrete supply / demand

In [ ]:
type_ = "supply"
discrete_supply_curve = dtuimsd.get_supply_demand_discrete_curve(
    type_, agg_supply_orders_df
)
discrete_supply_curve

In [ ]:
type_ = "demand"
discrete_demand_curve = dtuimsd.get_supply_demand_discrete_curve(
    type_, agg_demand_orders_df
)
discrete_demand_curve

In [ ]:
plt.plot(discrete_supply_curve)
plt.plot(discrete_demand_curve)
plt.show()